In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tools.data_cleansing import clean
from tools.data_exploration import print_email_by_id, emails_sent_distribution, emails_received_distribution,\
body_length_distribution, number_of_recipients_distribution
from tools.data_handling import enrich_emails, load_email_senders, unique_recipients, address_book, load_emails
from tools.evaluation import precision
from tools.features import split_tokenizer
from tools.sender_pipeline import SenderModel
from tools.submission import save_submission

# Data loading and first look

In [ ]:
df_emails = enrich_emails()

In [ ]:
print_email_by_id(df_emails, 41311)

# Data exploration

In [ ]:
df_email_senders = load_email_senders()

In [ ]:
emails_sent_distribution(df_email_senders, max_value=300)

In [ ]:
emails_received_distribution(df_emails, max_value=200)

In [ ]:
body_length_distribution(df_emails, max_value=10000)

In [ ]:
number_of_recipients_distribution(df_emails, bins=20, max_value=100)

In [ ]:
unique_rec = unique_recipients(df_emails)
print("Number of unique recipients: {}".format(len(unique_rec)))

# Data cleansing

## Cleansing function

In [ ]:
def clean_df(df_emails, df_senders):
    df_emails["clean body"] = ""
    for index, row in df_email_senders.iterrows():
        mids = list(map(int, row["mids"].split()))
        # data loading and separation
        df_interest = df_emails.ix[mids]
        # data cleansing
        unique_rec_train = unique_recipients(df_interest)
        add_book = address_book(unique_rec_train)
        df_emails.loc[mids, "clean body"] = df_interest["body"].apply(lambda x: clean(x, add_book))
    df_emails["clean body"] = df_emails["clean body"].fillna("")

In [ ]:
clean_df(df_emails, df_email_senders)

In [ ]:
df_emails.head()

# Training

In [ ]:
df_small_senders = df_email_senders.sample(frac=0.3)

## Current Model

In [ ]:
def model(df_interest, n_mails):
    # Input TF-IDF vectorizer
    input_bow = TfidfVectorizer(norm="l2")
    # Random Forest Regressor
    rf = RandomForestRegressor(
        n_estimators=15,
        max_depth=30,
        n_jobs=-1,
        min_samples_leaf=max(1,int(0.0002*n_mails)))
    # Output vectorizer
    unique_rec = unique_recipients(df_interest)
    output_bow = CountVectorizer(
        tokenizer=split_tokenizer,
        vocabulary=unique_rec)
    # Sender Model
    sender_model = SenderModel(
        df_emails=df_interest,
        classifier = rf,
        input_vectorizer = input_bow,
        output_vectorizer = output_bow
    )
    return sender_model

## On one particular sender

In [ ]:
row = df_email_senders.iloc[0]
mids = list(map(int, row["mids"].split()))
df_interest = df_emails.ix[mids]
n_mails = len(mids)
sender_model = model(df_interest, n_mails)
prec = sender_model.evaluate()
print(prec)

## Evaluation

In [ ]:
df_precision = pd.DataFrame(columns=["sender", "n_mails", "precision"])

In [ ]:
for index, row in df_small_senders.iterrows():
    # Row unpacking
    sender = row["sender"]
    mids = list(map(int, row["mids"].split()))
    df_interest = df_emails.ix[mids]
    n_mails = len(mids)
    # Model building
    sender_model = model(df_interest, n_mails)
    # Precision computation
    prec = sender_model.evaluate()
    df_precision.loc[index] = [sender, n_mails, prec]

In [ ]:
df_precision

In [ ]:
df_precision["precision"].mean()

# Submission

## Actual model training

In [ ]:
sender_models = dict()
for index, row in df_email_senders.iterrows():
    # Row unpacking
    sender = row["sender"]
    mids = list(map(int, row["mids"].split()))
    df_interest = df_emails.ix[mids]
    n_mails = len(mids)
    # Model building
    sender_model = model(df_interest, n_mails)
    # Training
    sender_model.train()
    # Saving
    sender_models[sender] = sender_model

## Data loading

In [ ]:
df_submission_senders = load_email_senders(set_type="test")
df_submission = load_emails(set_type="test")
df_submission["recipients"] = ""

## Data cleansing

In [ ]:
clean_df(df_submission, df_submission_senders)

## Prediction

In [ ]:
for index, row in df_submission_senders.iterrows():
    sender = row["sender"]
    mids = map(int, row["mids"].split())
    # Model loading
    sender_model = sender_models[sender]
    # Prediction
    sender_model.predict(mids, df_submission)

# Fine tuning

In [ ]:
from tools.fine_tuning import expected_precision

In [ ]:
depths = [2, 5, 10, 15, 20, 30, 50, 100, 300]
n = len(depths)
p = np.zeros(n)

In [ ]:
for i, depth in enumerate(depths):
    p[i] = expected_precision(min_sample_prop=0.0002,
                             n_estimators=15,
                             max_depth=100)

In [ ]:
n_trials = 10
pr = np.zeros(n_trials)
for i in range(n_trials):
    pr[i] += expected_precision(min_sample_prop=0.04,
                             n_estimators=15,
                             max_depth=30)
print(pr)

In [ ]:
pr.mean()

In [ ]:
plt.plot(depths, p)
plt.xlabel("Number of trees")
plt.ylabel("Precision")

## Formatting

In [ ]:
save_submission(df_submission,
               algo="RF single sender regressor tf idf",
               member="Zac")